In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
# Função para carregar o arquivo CSV
def load_data(file_path):
    df = pd.read_csv(file_path, delimiter=';', decimal=',', skipinitialspace=True, encoding='ISO-8859-1', skiprows=1)
    df.columns = [
        'Tempo (s)',
        'Deslocamento (mm)',
        'Força (kN)',
        'Deslocamento à tração (mm)',
        'Quantidade de PIP',
        'Tenacidade (N/tex)',
        'Tensão à tração (MPa)',
        'Deformação à tração (%)'
    ]

    # Converte apenas as colunas numéricas para float
    colunas_numericas = [
        'Tempo (s)', 
        'Deslocamento (mm)', 
        'Força (kN)', 
        'Deslocamento à tração (mm)', 
        'Tenacidade (N/tex)', 
        'Tensão à tração (MPa)', 
        'Deformação à tração (%)'
    ]
    for col in colunas_numericas:
        if df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].str.replace(',', '.'), errors='coerce')
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
    return df

# Caminho para o arquivo CSV
file_path = '../Lab1/PLA NORMAL/PLA NORMAL_1_1.csv'

# Carregar os dados
df = load_data(file_path)

# Ver os primeiros registros para conferir
print(df.head())


   Tempo (s)  Deslocamento (mm)  Força (kN)  Deslocamento à tração (mm)  \
0       0.02             0.6173      0.0001                      0.0000   
1       0.04             0.6172      0.0001                      0.0000   
2       0.06             0.6172      0.0001                     -0.0001   
3       0.08             0.6172      0.0001                     -0.0001   
4       0.10             0.6172      0.0001                     -0.0001   

   Quantidade de PIP  Tenacidade (N/tex)  Tensão à tração (MPa)  \
0                0.0              0.0518                 0.0009   
1                0.0              0.0572                 0.0010   
2                0.0              0.0751                 0.0013   
3                0.0              0.1043                 0.0017   
4                0.0              0.1274                 0.0021   

   Deformação à tração (%)  
0                      0.0  
1                      0.0  
2                      0.0  
3                      0.0  
4

In [145]:
df

,Tempo (s),Deslocamento (mm),Força (kN),Deslocamento à tração (mm),Quantidade de PIP,Tenacidade (N/tex),Tensão à tração (MPa),Deformação à tração (%)
0,0.0000,0.6173,0.0001,0.0000,0.0,0.0690,0.0012,0.0000
1,0.0200,0.6173,0.0001,0.0000,0.0,0.0518,0.0009,0.0000
2,0.0400,0.6172,0.0001,0.0000,0.0,0.0572,0.0010,0.0000
3,0.0600,0.6172,0.0001,-0.0001,0.0,0.0751,0.0013,0.0000
4,0.0800,0.6172,0.0001,-0.0001,0.0,0.1043,0.0017,0.0000
...,...,...,...,...,...,...,...,...
9209,183.4376,4.1391,1.3468,3.5219,0.0,1346.8063,22.4468,1.8536
9210,183.4380,4.1594,1.3227,3.5421,0.0,1322.7423,22.0457,1.8643
9211,183.4384,4.1796,1.2987,3.5623,0.0,1298.7085,21.6451,1.8749
9212,183.4388,4.1998,1.2748,3.5826,0.0,1274.7610,21.2460,1.8856


In [5]:
# 1 num_pontos= 183.38 / 0.02   num_pontos=    
# 2 num_pontos= 167.90 / 0.02   num_pontos=
# 3 num_pontos= 182.70 / 0.02   num_pontos=9135
# 4 num_pontos= 175.8 / 0.02    num_pontos=8790


from scipy import stats


def calc_modulo_elasticidade_dinamico(df, limite_erro=0.01): #como entrada um DataFrame df e um limite_erro, que determina quão próximo da linearidade os dados devem estar.
    # Filtra valores de tensão positivos
    filtro = df[df['Tensão à tração (MPa)'] > 0]

    # Inicializa variáveis
    n_max = len(filtro)
    modulo_elasticidade = np.nan

    for n in range(2, n_max):  # Começar com 2 pontos
        tensao = filtro['Tensão à tração (MPa)'].iloc[:n].values      #Para cada iteração do loop, os dados de tensão e deformação são extraídos até o índice n. A deformação é convertida de porcentagem para fração dividindo por 100.
        deformacao = filtro['Deformação à tração (%)'].iloc[:n].values / 100

        # Verifica se todos os valores de deformação ou tensão são iguais
        if np.all(deformacao == deformacao[0]) or np.all(tensao == tensao[0]):
            continue  # Ignora este conjunto e passa para o próximo

        # Realiza a regressão linear
        slope, intercept, r_value, p_value, std_err = stats.linregress(deformacao, tensao) #Realiza a regressão linear nos dados de tensão e deformação. 
                                                                                           #slope representa o coeficiente angular da linha de melhor ajuste, 
                                                                                           # que é o que queremos calcular como o módulo de elasticidade.

        # Prediz a tensão usando a linha de melhor ajuste
        tensao_predita = slope * deformacao + intercept

        # Prediz os valores de tensão com base na linha de melhor ajuste e calcula o Erro Quadrático Médio (EQM) entre os valores reais e os preditos.
        eqm = np.mean((tensao - tensao_predita) ** 2)


#Se o EQM for menor que o limite definido, atualiza modulo_elasticidade com o valor do coeficiente angular (slope).
        # Verifica se o EQM é menor que o limite
        if eqm < limite_erro:
            modulo_elasticidade = slope  # O coeficiente angular é o módulo de elasticidade
        else:
            break  # Sai do loop se a linearidade não se mantiver

    return modulo_elasticidade

# Exemplo de uso
modulo_elasticidade_dinamico = calc_modulo_elasticidade_dinamico(df)

if not np.isnan(modulo_elasticidade_dinamico):
    print(f"Módulo de Elasticidade (dinâmico): {modulo_elasticidade_dinamico:.2f} MPa")
else:
    print("Não foi possível calcular o módulo de elasticidade.")

Módulo de Elasticidade (dinâmico): 2985.99 MPa


In [6]:
def calc_resistencia_maxima(df):
    # Verifica se o DataFrame contém dados suficientes
    if df.empty or 'Tensão à tração (MPa)' not in df.columns:
        return np.nan, np.nan, np.nan  # Retorna NaN se não houver dados suficientes

    # Encontra a resistência máxima e a deformação correspondente
    resistencia_maxima = df['Tensão à tração (MPa)'].max()  # Encontra a máxima tensão
    idx_max = df['Tensão à tração (MPa)'].idxmax()  # Índice da máxima tensão
    deformacao_maxima_percent = df.loc[idx_max, 'Deformação à tração (%)']  # Deformação correspondente à tensão máxima
    deformacao_maxima_mm = df.loc[idx_max, 'Deslocamento à tração (mm)']  # Deformação em mm

    return resistencia_maxima, deformacao_maxima_percent, deformacao_maxima_mm

# Cálculo da resistência máxima e deformações correspondentes
resistencia_maxima, deformacao_maxima, deformacao_maxima_mm = calc_resistencia_maxima(df)

# Resultados
print(f"Resistência Máxima: {resistencia_maxima:.2f} MPa")
print(f"Deformação na Resistência Máxima: {deformacao_maxima:.2f} %")
print(f"Deformação na Resistência Máxima (mm): {deformacao_maxima_mm:.2f} mm")

Resistência Máxima: 35.35 MPa
Deformação na Resistência Máxima: 1.60 %
Deformação na Resistência Máxima (mm): 3.03 mm


In [7]:
def calc_resistencia_ruptura(df):
    # Verifica se o DataFrame contém dados suficientes
    if df.empty or 'Tensão à tração (MPa)' not in df.columns:
        return np.nan, np.nan  # Retorna NaN se não houver dados suficientes

    # A resistência na ruptura é a última tensão registrada no DataFrame
    resistencia_ruptura = df['Tensão à tração (MPa)'].iloc[-1]  # Última tensão
    deformacao_ruptura = df['Deslocamento à tração (mm)'].iloc[-1]  # Último valor de deslocamento

    return resistencia_ruptura, deformacao_ruptura

# Cálculos
resistencia_ruptura, deformacao_ruptura_mm = calc_resistencia_ruptura(df)

# Resultados
print(f"Resistência na Ruptura: {resistencia_ruptura:.2f} MPa")
print(f"Deformação na Ruptura: {deformacao_ruptura_mm:.2f} mm")


Resistência na Ruptura: 21.05 MPa
Deformação na Ruptura: 3.59 mm


In [8]:
def calc_tenacidade_j_m3(df):
    # Verifica se o DataFrame contém dados suficientes
    if df.empty or 'Tensão à tração (MPa)' not in df.columns or 'Deformação à tração (%)' not in df.columns:
        return np.nan  # Retorna NaN se não houver dados suficientes

    # Filtra dados válidos
    df = df[(df['Tensão à tração (MPa)'] > 0) & (df['Tensão à tração (MPa)'].notna())]

    if df.empty:
        return np.nan  # Retorna NaN se não houver dados válidos

    # Convertendo a tensão de MPa para Pa
    tensao_pa = df['Tensão à tração (MPa)'] * 1e6
    print("Tensão (Pa):")
    print(tensao_pa.dropna())  # Mostra apenas os valores não nulos

    # Convertendo a deformação de porcentagem para fração
    deformacao_fraçao = df['Deformação à tração (%)'] / 100  
    print("Deformação (fração):")
    print(deformacao_fraçao.dropna())  # Mostra apenas os valores não nulos

    # Cálculo da tenacidade como a área sob a curva
    tenacidade = np.trapezoid(tensao_pa, deformacao_fraçao)  # Área sob a curva

    return tenacidade

# Chame a função e imprima a tenacidade
tenacidade_j_m3 = calc_tenacidade_j_m3(df)
print(f"Tenacidade: {tenacidade_j_m3:.2f} J/m³")


Tensão (Pa):
0            900.0
1           1000.0
2           1300.0
3           1700.0
4           2100.0
           ...    
9208    22446800.0
9209    22045700.0
9210    21645100.0
9211    21246000.0
9212    21047200.0
Name: Tensão à tração (MPa), Length: 9213, dtype: float64
Deformação (fração):
0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
9208    0.018536
9209    0.018643
9210    0.018749
9211    0.018856
9212    0.018909
Name: Deformação à tração (%), Length: 9213, dtype: float64
Tenacidade: 411953.86 J/m³
